In [ ]:
pip install numpy scipy pythran bitarray

     |████████████████████████████████| 4.3 MB 23.5 MB/s 
     |████████████████████████████████| 88 kB 3.9 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
  Created wheel for bitarray: filename=bitarray-2.3.4-cp37-cp37m-linux_x86_64.whl size=171944 sha256=9121b1ff44f7f87ee5202f1d7df46445018cbab07da398dbb9607d1f6d5eadc5
  Stored in directory: /root/.cache/pip/wheels/84/cc/5b/0e861bdb5294d22d2d4f595df936f964a95258387e11494d41
Successfully built bitarray
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires gast==0.4.0, but you have gast 0.5.2 which is incompatible.


In [ ]:
!git clone https://github.com/ChalmersPhotonicsLab/QAMpy.git

Cloning into 'QAMpy'...
remote: Enumerating objects: 9866, done.
remote: Counting objects: 100% (1605/1605), done.
remote: Compressing objects: 100% (519/519), done.
remote: Total 9866 (delta 1190), reused 1479 (delta 1083), pack-reused 8261
Receiving objects: 100% (9866/9866), 225.76 MiB | 32.33 MiB/s, done.
Resolving deltas: 100% (6781/6781), done.


In [ ]:
cd QAMpy/

/content/QAMpy


In [ ]:
!python setup.py build

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/qampy
copying qampy/filtering.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/analog_frontend.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/phaserec.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/helpers.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/impairments.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/theory.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/__init__.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/equalisation.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/signals.py -> build/lib.linux-x86_64-3.7/qampy
copying qampy/io.py -> build/lib.linux-x86_64-3.7/qampy
creating build/lib.linux-x86_64-3.7/qampy/core
copying qampy/core/segmentaxis.py -> build/lib.linux-x86_64-3.7/qampy/core
copying qampy/core/filter.py -> build/lib.linux-x86_64-3.7/qampy/core
copying qampy/core/ber_functions.py -> build/lib.linux-x86_

In [ ]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating qampy.egg-info
writing qampy.egg-info/PKG-INFO
writing dependency_links to qampy.egg-info/dependency_links.txt
writing requirements to qampy.egg-info/requires.txt
writing top-level names to qampy.egg-info/top_level.txt
writing manifest file 'qampy.egg-info/SOURCES.txt'
adding license file 'COPYING'
writing manifest file 'qampy.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/qampy
copying build/lib.linux-x86_64-3.7/qampy/filtering.py -> build/bdist.linux-x86_64/egg/qampy
copying build/lib.linux-x86_64-3.7/qampy/analog_frontend.py -> build/bdist.linux-x86_64/egg/qampy
copying build/lib.linux-x86_64-3.7/qampy/phaserec.py -> build/bdist.linux-x86_64/egg/qampy
creating build/bdist.linux-x86_64/egg/qampy/core
copying build/lib.linux-x86_64-3

In [ ]:
from Funcoes import *
from tensorflow.keras.models import load_model
from scipy.interpolate import interp1d

In [ ]:
%matplotlib inline

In [ ]:
plt.rcParams['font.size'] = 18
plt.rcParams['figure.figsize'] = [16, 8]
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['lines.markersize'] = 10

In [ ]:
M = 16        # ordem da modulação
Fb = 24e9      # taxa de símbolos
SpS = 2         # amostras por símbolo
Fs = SpS*Fb    # taxa de amostragem
rolloff = 0.05  # Rolloff do filtro formatador de pulso
amostras = 512
size = 110000
def data_test_to_obtain_BERxSNR_plot(model,data,X):
    X = MinMaxScaler().fit_transform(X)
    predictions = model.predict(X)
    predicted_signal = data['amplitudes']*np.exp(1j*predictions.reshape(-1,))
    return predicted_signal.reshape((1,-1))

In [ ]:
bers_of_model = {}
SNRs = np.arange(18,49,2)
SpS = 8
M = 64
amostras = 256
for CSPR in range(4,17):
  model = load_model(f'model_{SpS}_sps_256_amostras_{CSPR}_CSPR.h5')
  BERs = np.zeros(SNRs.size)
  for i , SNR in enumerate(SNRs):
      qam_signal = generate_signal(M,Fb,SpS,SNR,rolloff=0.05)
      A = np.sqrt(2*CSPR*np.mean(np.abs(qam_signal**2)))
      sfm, _,_ = qam_signal_phase_min(qam_signal,A)
      data,X,y = dataset_02(sfm,amostras,size)
      predicted_signal = data_test_to_obtain_BERxSNR_plot(model,data,X)
      reverted_signal = revert_sfm(predicted_signal,A)
      BERs[i] = reverted_signal[:,5000:110000-5000:SpS].cal_ber()
  bers_of_model[f'model_{SpS}_sps_256_amostras_{CSPR}_CSPR_64QAM'] = BERs
  print(bers_of_model)

{'model_8_sps_256_amostras_4_CSPR_64QAM': array([0.11553333, 0.10332   , 0.08728   , 0.08070667, 0.07405333,
       0.07013333, 0.06744   , 0.06674667, 0.06474667, 0.06376   ,
       0.06436   , 0.06634667, 0.06453333, 0.06592   , 0.06368   ,
       0.0648    ])}
{'model_8_sps_256_amostras_4_CSPR_64QAM': array([0.11553333, 0.10332   , 0.08728   , 0.08070667, 0.07405333,
       0.07013333, 0.06744   , 0.06674667, 0.06474667, 0.06376   ,
       0.06436   , 0.06634667, 0.06453333, 0.06592   , 0.06368   ,
       0.0648    ]), 'model_8_sps_256_amostras_5_CSPR_64QAM': array([0.08337333, 0.05864   , 0.03888   , 0.02864   , 0.01772   ,
       0.01572   , 0.00862667, 0.00544   , 0.00925333, 0.00736   ,
       0.00797333, 0.00692   , 0.00676   , 0.00616   , 0.00925333,
       0.00696   ])}
{'model_8_sps_256_amostras_4_CSPR_64QAM': array([0.11553333, 0.10332   , 0.08728   , 0.08070667, 0.07405333,
       0.07013333, 0.06744   , 0.06674667, 0.06474667, 0.06376   ,
       0.06436   , 0.06634667, 0.

In [ ]:
 bers_of_model

{'model_8_sps_256_amostras_10_CSPR_64QAM': array([0.07974667, 0.05390667, 0.03386667, 0.02310667, 0.01318667,
        0.00812   , 0.00717333, 0.00601333, 0.00449333, 0.00321333,
        0.00378667, 0.00452   , 0.00244   , 0.00346667, 0.00390667,
        0.00368   ]),
 'model_8_sps_256_amostras_11_CSPR_64QAM': array([0.08012   , 0.05505333, 0.03309333, 0.02277333, 0.01432   ,
        0.00844   , 0.00561333, 0.00553333, 0.00518667, 0.00466667,
        0.00468   , 0.0042    , 0.00469333, 0.00542667, 0.00362667,
        0.00386667]),
 'model_8_sps_256_amostras_12_CSPR_64QAM': array([0.07918667, 0.05486667, 0.03484   , 0.025     , 0.01325333,
        0.01046667, 0.007     , 0.00668   , 0.00405333, 0.00302667,
        0.00368   , 0.00448   , 0.00344   , 0.00572   , 0.00274667,
        0.00422667]),
 'model_8_sps_256_amostras_13_CSPR_64QAM': array([0.08068   , 0.05366667, 0.0354    , 0.02112   , 0.01557333,
        0.00914667, 0.00938667, 0.00453333, 0.00432   , 0.00444   ,
        0.004     

In [ ]:
import pickle
a_file = open(f"BER_model_{SpS}_sps_256_amostras_64QAM.pkl", "wb")
pickle.dump(bers_of_model, a_file)
a_file.close()